In [1]:
import os
import csv
import numpy as np
import random
import tensorly as tl
from tensorly.decomposition import non_negative_parafac
import scipy.sparse
import sys
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [2]:
def Qmat(S,configuration,path):
    dirname = os.fsdecode(configuration)
    filename = os.path.join(path, dirname+'/coords.csv_sparse_graph.npz')
    if os.path.isfile(filename): 
        A = scipy.sparse.load_npz(filename)
        '''construct modularity matrix'''
        M = A
        k = A.sum(axis=0)
        w = A.sum(axis=None)
        M = A - np.outer(k,k)*0.5/w
        output = M.shape[0]*M.shape[1]*np.trace(np.dot(np.dot(S.transpose(),M),S))/(2.0*w) #rescale by network size
    return output

def sample_modularity(S,cf_samples,path):
    modularity_values = []
    for configuration in cf_samples:
            modularity_values.append(Qmat(S,configuration,path))  
    return modularity_values

def random_community(Sp,configuration,path):
    np.random.shuffle(Sp)
    return Qmat(Sp,configuration,path)

def membership(factors):
    S = np.zeros(shape=factors[1][1].shape)
    for c in range(S.shape[1]):
        vec = 0.5*(factors[1][1][:,c]+factors[1][2][:,c]) # take the average of the 2 factors, that should be identical
        S[:,c] = vec/tl.norm(vec,1) #normalize membership
    Sp = np.copy(S) # this copy can be used for significance testing
    return S,Sp

In [3]:
samples = 10 #numb of sampled 3d structured
path = '/media/garner1/hdd1/gpseq/10000'
configurations = os.listdir(path) 
config_sample = random.sample(configurations, k=samples) # sample k times without replacement from configurations

In [4]:
rank = 3 # numb of components in svd. The greater this value the slower the parafac convergence 
comm = 3 # numb of communities to retrive

In [ ]:
T = np.zeros(shape = (samples, 3043, 3043), dtype = np.float32)
graph_idx = 0
for config in config_sample:
    dirname = os.fsdecode(config)
    filename = os.path.join(path, dirname+'/coords.csv_tsvd.npz')
    if os.path.isfile(filename): 
        svd = np.load(filename)
        u = svd['u'][:,:rank]
        s = svd['s'][:rank]
        vt = svd['vt'][:rank,:]
        T[graph_idx,:,:] = np.dot(np.dot(u,np.diag(s)),vt)
        del svd
        continue
    else:
        T[graph_idx,:,:] = np.zeros(shape = (3043,3043),)
        continue
    graph_idx += 1

print(T.shape)   
factors = non_negative_parafac(T, rank=comm, verbose=1, n_iter_max=20,tol=1e-08,init='svd')
[factors[1][ind].shape for ind in range(3)]

In [ ]:
S, Sp = membership(factors)

In [ ]:
h1 = sample_modularity(S,config_sample,path) # S on the training data

In [ ]:
h2 = sample_modularity(S,random.sample(os.listdir(path), k=100),path) # S on the test data
mu_test = np.mean(h2); sigma_test = np.std(h2)

In [ ]:
ind = 1
h3 = [random_community(Sp,config_sample[ind],path) for count in range(5) for ind in range(samples)] # random S on one of the training data
mu_null = np.mean(h3); sigma_null = np.std(h3)

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})
labels = ['training data','test data', 'null model']
histos = [h1,h2,h3]
fig, ax = plt.subplots()
for count in range(3):
#     sns.distplot(histos[count], kde=False,norm_hist=True,label=labels[count],hist_kws=dict(alpha=0.7))
    sns.distplot(histos[count], rug=True, hist=False,label=labels[count])
    

ref = Qmat(S,config_sample[ind],path)
# ax.axvline(ref,color='red',label='z-score: '+str(np.round((ref-mu)/sigma))) # draw a red vertical line at the value of S for the example graph
plt.legend()
plt.title('HiC only with model significance '+str(np.round((mu_test-mu_null)/(sigma_test+sigma_null))))
# print('z-score is: '+str((ref-mu)/sigma))  # z-score for a random S as the null model

In [ ]:
for i in range(comm):
    a = factors[1][1][:,i]
    b = factors[1][2][:,i]
    c = factors[1][0][:,i]
    mat = np.outer(0.5*(a+b),0.5*(a+b)) # symmetrize wrt a & b
    print(tl.norm(a,2)*tl.norm(b,2)*tl.norm(c,2))
#     plt.imshow(mat, cmap='Blues', interpolation='nearest')
    plt.figure(figsize=(10, 10))
    sns.heatmap(mat,cmap='Blues',square=True,xticklabels=False,yticklabels=False)
    plt.show()  
      

In [ ]:
ind = 3
dirname = os.fsdecode(config_sample[ind])
filename = os.path.join(path, dirname+'/coords.csv_sparse_graph.npz')
A = scipy.sparse.load_npz(filename).todense()
sns.heatmap(A,cmap='Blues',square=True,xticklabels=False,yticklabels=False)

In [ ]:
for i in range(comm):
    mat = np.outer(S[:,i],S[:,i]) # symmetrize wrt a & b
    plt.figure(figsize=(10, 10))
    sns.heatmap(np.round(mat.shape[0]*mat.shape[1]*np.multiply(mat,A),decimals=3),square=True,xticklabels=False,yticklabels=False)
    plt.show()  